In [124]:
import pandas as pd

In [125]:
df = pd.read_csv('Test.csv')
# df = df.rename(columns={'Content': 'Text Content'})
df = df.rename(columns={'Avg_rating': 'Avg rating'})

In [126]:
from afinn import Afinn
afinn = Afinn()

def calculate_normalized_sentiment(text):
    words = text.split()
    if len(words) > 0:
        return afinn.score(text) / len(words)
    else:
        return 0

def calculate_lexical_diversity(text):
    text = str(text)
    
    words = text.split()
    
    unique_words = set(words)
    lexical_diversity = len(unique_words) / len(words) if len(words) > 0 else 0
    
    return lexical_diversity

lexical_diversities = []

for index, row in df.iterrows():
    joke_text = row['Text Content']
    
    lexical_diversity = calculate_lexical_diversity(joke_text)
    lexical_diversities.append({'Lexical Diversity': lexical_diversity})

lexical_diversity_df = pd.DataFrame(lexical_diversities)



def calculate_punctuation_frequency(text):
    text = str(text)
    
    punctuation_marks = ['!', '?']
    
    punctuation_counts = {mark: text.count(mark) for mark in punctuation_marks}
    
    return punctuation_counts

punctuation_frequencies = []

for index, row in df.iterrows():
#     joke_id = row['joke_ids']
    joke_text = row['Text Content']
    
    punctuation_frequency = calculate_punctuation_frequency(joke_text)
#     punctuation_frequency['joke_id'] = joke_id
    punctuation_frequencies.append(punctuation_frequency)

punctuation_df = pd.DataFrame(punctuation_frequencies)

punctuation_df

,!,?
0,0,1
1,0,1
2,0,1
3,0,0
4,0,2
...,...,...
135,1,4
136,2,0
137,1,2
138,0,1


In [127]:
from nltk.tokenize import word_tokenize


df['Joke_len'] = df['Text Content'].str.len()
df['sentiment_score'] = df['Text Content'].apply(calculate_normalized_sentiment)
df = pd.concat([df, lexical_diversity_df], axis=1)
df = pd.concat([df, punctuation_df], axis=1)
df['Num_Words'] = df['Text Content'].apply(lambda x: len([word for word in word_tokenize(x) if word.isalpha()]))


In [128]:
df

,Unnamed: 0,Text Content,Avg_Rating,Joke_len,sentiment_score,Lexical Diversity,!,?,Num_Words
0,4,Q.\tWhat's O. J. Simpson's Internet address? ...,-2.034618,94,-0.642857,0.857143,0,1,12
1,6,How many feminists does it take to screw in a ...,-1.943754,75,0.266667,1.000000,0,1,15
2,7,Q. Did you hear about the dyslexic devil worsh...,-0.704781,83,0.000000,1.000000,0,1,16
3,12,They asked the Japanese visitor if they have e...,-0.671219,100,0.000000,1.000000,0,0,16
4,14,Q: What did the blind person say when given s...,-1.436065,87,-0.294118,0.941176,0,2,17
...,...,...,...,...,...,...,...,...,...
135,145,America: 8:00 - Welcome to work! 12:00 - Lunch...,0.205030,311,0.047619,0.571429,1,4,43
136,146,It was the day of the big sale. Rumors of the ...,1.782516,592,0.032787,0.672131,2,0,121
137,147,"Recently a teacher, a garbage collector, and a...",2.968233,795,0.007042,0.711268,1,2,136
138,148,"A little girl asked her father, ""Daddy? Do all...",2.394606,185,0.000000,0.857143,0,1,33


In [129]:
# import pickle

# with open('linear_regression_model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)


In [130]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

X = df.iloc[:, 3:]
y_test = df.iloc[:, 2]

# y_pred = loaded_model.predict(X)
# mse = mean_squared_error(y_test, y_pred)
# r_squared = r2_score(y_test, y_pred)

In [131]:
# r_squared

In [132]:
import pickle

with open('ensemble_model.pkl', 'rb') as model_file:
    (bagged_linear_reg, bagged_svm_linear, bagged_ridge_reg, bagged_lasso_reg) = pickle.load(model_file)



In [134]:
import pickle
from sklearn.metrics import mean_squared_error, r2_score

with open('ensemble_model.pkl', 'rb') as model_file:
    ensemble_models = pickle.load(model_file)

bagged_linear_reg, bagged_svm_linear, bagged_ridge_reg, bagged_lasso_reg = ensemble_models

X_test = X  # Replace with your test data

y_pred_linear_reg = bagged_linear_reg.predict(X_test)
y_pred_svm_linear = bagged_svm_linear.predict(X_test)
# y_pred_ridge_reg = bagged_ridge_reg.predict(X_test)
# y_pred_lasso_reg = bagged_lasso_reg.predict(X_test)

ensemble_predictions = (y_pred_linear_reg + y_pred_svm_linear) / 2

mse = mean_squared_error(y_test, ensemble_predictions)
r_squared = r2_score(y_test, ensemble_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)


Mean Squared Error: 1.6037377451373767
R-squared: 0.2016590102708573


In [122]:
X_test

,Joke_len,sentiment_score,Lexical Diversity,!,?,Num_Words
0,94,-0.642857,0.857143,0,1,12
1,75,0.266667,1.000000,0,1,15
2,83,0.000000,1.000000,0,1,16
3,100,0.000000,1.000000,0,0,16
4,87,-0.294118,0.941176,0,2,17
...,...,...,...,...,...,...
135,311,0.047619,0.571429,1,4,43
136,592,0.032787,0.672131,2,0,121
137,795,0.007042,0.711268,1,2,136
138,185,0.000000,0.857143,0,1,33
